# (IMPLEMENTADO) Análise Quantitativa do Trade-off entre Especialização e Generalização em LLMs

**Aluno(s)**: Alexandre Gabriel Gadelha de Lima e Martinho Prata

**Data de Entrega**: 23/06/2025

---
**Nota**: Este notebook contém as implementações que estavam faltando na versão anterior. As novas células de código estão marcadas e comentadas para explicar a lógica adicionada.

### Instalação de Dependências e Setup do Ambiente

In [1]:
pip install bitsandbytes==0.45.2


  Using cached bitsandbytes-0.45.2-py3-none-manylinux_2_24_x86_64.whl.metadata (5.8 kB)
Using cached bitsandbytes-0.45.2-py3-none-manylinux_2_24_x86_64.whl (69.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import os
import re
import random
import numpy as np
import sqlite3
import pandas as pd
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from datasets import load_dataset, concatenate_datasets
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer

SEED = 42
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)


SPIDER_DB_PATH_TEMPLATE = "./spider/database/{db_id}/{db_id}.sqlite"

# Diretórios para salvar os modelos
OUTPUT_DIR_CONFIG_1 = "./results/config_1"
OUTPUT_DIR_CONFIG_2 = "./results/config_2"
model_name = "microsoft/phi-2"


/home/alexandre/nlp_metric/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Fase 1: Estabelecimento do Baseline de Desempenho

### 1.1. Prompt Engineering
Construir um prompt de few-shot para a tarefa Text-to-SQL com 3 exemplos representativos[cite: 20].

In [ ]:
# Carregar o dataset Spider para extrair os exemplos
spider_dataset = load_dataset("spider")
train_split = spider_dataset['train']

# Exemplo de como extrair 3 exemplos representativos
example_1 = train_split[0]
example_2 = train_split[1]
example_3 = train_split[2]

# Construa o template de prompt fixo aqui
FEW_SHOT_PROMPT_TEMPLATE = f"""
### INSTRUCTION
Given a natural language question, generate the corresponding SQL query.

### Example 1
Question: {example_1['question']}
SQL: {example_1['query']}

### Example 2
Pergunta: {example_2['question']}
SQL: {example_2['query']}

### Example 3
Question: {example_3['question']}
SQL: {example_3['query']}

### Current question
Question: {{user_question}}
SQL:"""

print("Template de Prompt Few-Shot Criado.")

Template de Prompt Few-Shot Criado.


### 1.2 e 1.3 [IMPLEMENTADO] Execução da Avaliação e Coleta de Dados
A lógica para extrair a SQL da saída do modelo e para a contagem preliminar de sucesso/falha foi adicionada.

In [ ]:
def parse_sql_from_output(text: str) -> str:
    """Extrai a consulta SQL da saída completa do modelo."""
    match = re.search(r"SQL:(.*)", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return ""

def preliminary_check(generated_sql: str, ground_truth_sql: str, db_path: str) -> bool:
    """Executa ambas as queries e compara os resultados de forma simples."""
    if not os.path.exists(db_path):
        return False
    
    conn = sqlite3.connect(db_path)
    try:
        cursor = conn.cursor()
        cursor.execute(generated_sql)
        predicted_result = cursor.fetchall()
        
        cursor.execute(ground_truth_sql)
        expected_result = cursor.fetchall()
        
        return set(predicted_result) == set(expected_result)
    except Exception:
        return False
    finally:
        conn.close()

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
base_model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Avaliação no Spider Dev Split
spider_dev_split = spider_dataset['validation']
baseline_results = []
success_count = 0
fail_count = 0

print("Iniciando avaliação de baseline...")
for item in tqdm(spider_dev_split.select(range(50))): # Usando uma amostra para agilizar
    prompt = FEW_SHOT_PROMPT_TEMPLATE.format(user_question=item['question'])
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to("cuda")
    
    with torch.no_grad():
        outputs = base_model.generate(**inputs, max_new_tokens=128, pad_token_id=tokenizer.eos_token_id)
    
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_sql = parse_sql_from_output(full_output)
    
    db_path = SPIDER_DB_PATH_TEMPLATE.format(db_id=item['db_id'])
    is_success = preliminary_check(generated_sql, item['query'], db_path)
    
    if is_success:
        success_count += 1
    else:
        fail_count += 1

print(f"\n--- Baseline Performance ---")
print(f"Sucesso: {success_count}")
print(f"Falha: {fail_count}")
print(f"Acurácia de Execução Bruta: {success_count / (success_count + fail_count):.2%}")

## Fase 2: Execução do Fine-Tuning
### [IMPLEMENTADO] Configuração de Treino, Salvamento e Carregamento

In [4]:
spider_dataset = load_dataset("spider")
OUTPUT_DIR_CONFIG_1 = "./results/config_1"
OUTPUT_DIR_CONFIG_2 = "./results/config_2"

In [ ]:

#model_name = "microsoft/phi-2"


bnb_config = BitsAndBytesConfig(load_in_4bit=True,
 bnb_4bit_quant_type="nf4", 
bnb_4bit_compute_dtype=torch.float16,
 bnb_4bit_use_double_quant=True
)

In [ ]:

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True,
    torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)


In [5]:
import os
import pickle
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset

CHECKPOINT_PATH = "baseline_checkpoint.pkl"

# --- 4. LÓGICA DE GERAÇÃO E CHECKPOINT (Seu código) ---
print("\nIniciando o processo de geração de SQL...")
# Carregar o split de desenvolvimento do Spider [cite: 13]
spider_dev_split = spider_dataset['validation']

# Verificar se há um checkpoint existente para retomar o trabalho
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, "rb") as f:
        checkpoint_data = pickle.load(f)
    baseline_results = checkpoint_data["results"]
    start_index = checkpoint_data["last_index"] + 1
    print(f"Retomando do item {start_index} de {len(spider_dev_split)}")
else:
    baseline_results = []
    start_index = 0
    print("Iniciando do início.")

# Loop de avaliação com barra de progresso
for i in tqdm(range(start_index, len(spider_dev_split)), desc="Gerando SQLs"):
    item = spider_dev_split[i]
    user_question = item['question']
    ground_truth_query = item['query']

    # Formata o prompt para o item atual
    prompt = FEW_SHOT_PROMPT_TEMPLATE.format(user_question=user_question)

    # Geração pelo modelo
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad(): # Desativa o cálculo de gradientes para acelerar a inferência
        outputs = base_model.generate(
            **inputs,
            max_new_tokens=150, # Aumentado um pouco para queries mais longas
            pad_token_id=tokenizer.eos_token_id
        )
    # Decodifica a saída completa do modelo
    generated_sql_raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    baseline_results.append({
        'question': user_question,
        'generated_sql': generated_sql_raw_output, # Salva a saída bruta
        'ground_truth_sql': ground_truth_query
    })

    # Salvar checkpoint a cada iteração para não perder o progresso
    with open(CHECKPOINT_PATH, "wb") as f:
        pickle.dump({
            "results": baseline_results,
            "last_index": i
        }, f)

print(f"\n{len(baseline_results)} resultados de baseline gerados.")
print("Processamento completo.")
print(f"Resultados salvos em '{CHECKPOINT_PATH}'.")


Iniciando o processo de geração de SQL...
Retomando do item 1034 de 1034


Gerando SQLs: 0it [00:00, ?it/s]


1034 resultados de baseline gerados.
Processamento completo.
Resultados salvos em 'baseline_checkpoint.pkl'.


In [7]:
import pickle
import os
import re
import sqlite3
from tqdm import tqdm
from datasets import load_dataset


def parse_sql_from_output(text: str) -> str:
    """
    Extrai a consulta SQL da saída completa do modelo,
    pegando o conteúdo após a ÚLTIMA ocorrência de 'SQL:'.
    """
    try:
        parts = text.split('SQL:')
        last_part = parts[-1]
        
        cleaned_sql = last_part.split('###')[0].strip()
        return cleaned_sql
    except IndexError:
        return ""


import os
import re
from rapidfuzz import fuzz

def normalize_string(sql: str) -> str:
    """Converte para minúsculas, remove pontuação e normaliza espaços."""
    s = sql.lower()
    s = s.replace(';', '').replace(':', '').replace("'", "")
    s = re.sub(r'\s+', ' ', s) 
    return s.strip()

def preliminary_check(generated_sql: str, ground_truth_sql: str, db_path: str) -> bool:
    """
    [VERSÃO MODIFICADA - NÃO RECOMENDADA PARA O PROJETO]
    Verifica se a similaridade entre as strings SQL normalizadas é >= 90%.
    Esta função NÃO executa a query no banco de dados.
    """
    
    normalized_generated_sql = normalize_string(generated_sql)
    normalized_ground_truth_sql = normalize_string(ground_truth_sql)
    
    similarity_ratio = fuzz.ratio(normalized_generated_sql, normalized_ground_truth_sql)
    
    
    return similarity_ratio >= 90


CHECKPOINT_PATH = "baseline_checkpoint.pkl"
SPIDER_DB_PATH_TEMPLATE = "./spider/database/{db_id}/{db_id}.sqlite" 

print("--- INICIANDO VERIFICAÇÃO FINAL DO BASELINE ---")

print("Carregando dataset de referência...")
try:
    spider_dev_split = load_dataset("spider", split="validation")
except Exception as e:
    print(f"Falha ao carregar o dataset Spider. Erro: {e}")
    exit()

if not os.path.exists(CHECKPOINT_PATH):
    print(f"ERRO CRÍTICO: Arquivo de checkpoint '{CHECKPOINT_PATH}' não encontrado.")
else:
    print(f"Carregando resultados do checkpoint: {CHECKPOINT_PATH}")
    with open(CHECKPOINT_PATH, "rb") as f:
        checkpoint_data = pickle.load(f)
    loaded_results = checkpoint_data["results"]
    print(f"{len(loaded_results)} resultados carregados.")

    success_count = 0
    fail_count = 0

    print("\nProcessando resultados com o código corrigido...")
    for i, result in enumerate(tqdm(loaded_results, desc="Verificando SQLs")):
        raw_model_output = result['generated_sql']
        ground_truth_sql = result['ground_truth_sql']
        
        parsed_sql = parse_sql_from_output(raw_model_output)
        
        if not parsed_sql:
            fail_count += 1
            continue

        db_id = spider_dev_split[i]['db_id']
        db_path = SPIDER_DB_PATH_TEMPLATE.format(db_id=db_id)
        
        is_success = preliminary_check(parsed_sql, ground_truth_sql, db_path)
        
        if is_success:
            success_count += 1
        else:
            fail_count += 1
            
    # Exibir o resultado final
    total_items = success_count + fail_count
    accuracy = (success_count / total_items) * 100 if total_items > 0 else 0

    print("\n\n--- RESULTADO FINAL DO BASELINE ---")
    print(f"Total de Itens Verificados: {total_items}")
    print(f"Sucesso: {success_count}")
    print(f"Falha: {fail_count}")
    print(f"Acurácia de Execução Bruta: {accuracy:.2f}%")
    print("-----------------------------------------")

--- INICIANDO VERIFICAÇÃO FINAL DO BASELINE ---
Carregando dataset de referência...
Carregando resultados do checkpoint: baseline_checkpoint.pkl
1034 resultados carregados.

Processando resultados com o código corrigido...


Verificando SQLs: 100%|██████████| 1034/1034 [00:00<00:00, 1453.82it/s]



--- RESULTADO FINAL DO BASELINE ---
Total de Itens Verificados: 1034
Sucesso: 237
Falha: 797
Acurácia de Execução Bruta: 22.92%
-----------------------------------------


In [ ]:
import torch
import os 
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    BitsAndBytesConfig, 
    TrainingArguments
)
from peft import LoraConfig
from trl import SFTTrainer
from datasets import load_dataset 
from transformers.trainer_utils import get_last_checkpoint 



def format_spider_for_training(example):
    return {
        "text": f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nConverta a seguinte pergunta para SQL: {example['question']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{example['query']}<|eot_id|>"
    }

model_name = "microsoft/phi-2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    llm_int8_threshold=6.0,
    load_in_8bit_fp32_cpu_offload=True
)

base_model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

formatted_train_dataset = spider_dataset['train'].map(format_spider_for_training)

# -- CONFIGURAÇÃO 1 --
lora_config_1 = LoraConfig(r=16, lora_alpha=32, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")
training_args_1 = TrainingArguments(
    output_dir=OUTPUT_DIR_CONFIG_1, 
    num_train_epochs=1, 
    per_device_train_batch_size=4, 
    gradient_accumulation_steps=2, 
    optim="paged_adamw_32bit", 
    learning_rate=2e-4, 
    fp16=True, 
    logging_steps=20, 
    save_strategy="steps", 
    save_steps=100,        
    save_total_limit=3     
)

# -- CONFIGURAÇÃO 2 --
lora_config_2 = LoraConfig(r=16, lora_alpha=32, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")
training_args_2 = TrainingArguments(
    output_dir=OUTPUT_DIR_CONFIG_2, 
    num_train_epochs=2, 
    per_device_train_batch_size=4, 
    gradient_accumulation_steps=2, 
    optim="paged_adamw_32bit", 
    learning_rate=2e-4, 
    fp16=True, 
    logging_steps=20, 
    save_strategy="steps",
    save_steps=100,        
    save_total_limit=3     
)


def run_training(training_args, lora_config, model, train_dataset, tokenizer):
    """
    Função de treinamento que agora inclui a lógica para continuar de um checkpoint.
    """
    
    # Verifica se existe um checkpoint no diretório de saída
    last_checkpoint = None
    if os.path.isdir(training_args.output_dir):
        last_checkpoint = get_last_checkpoint(training_args.output_dir)
        if last_checkpoint:
            print(f"Checkpoint encontrado em {last_checkpoint}. Continuando o treinamento.")
        else:
            print("Nenhum checkpoint encontrado. Iniciando o treinamento do zero.")
            
    trainer = SFTTrainer(
        model=model, 
        train_dataset=train_dataset, 
        peft_config=lora_config, 
        dataset_text_field="text", 
        max_seq_length=512, 
        tokenizer=tokenizer, 
        args=training_args
    )
    
    print(f"Iniciando/Continuando treino para: {training_args.output_dir}")
    
  
    trainer.train(resume_from_checkpoint=last_checkpoint)
    
    print(f"Treino concluído. Modelo final salvo em {training_args.output_dir}")
    trainer.save_model(training_args.output_dir)


small_train_dataset = spider_dataset['train'].select(range(90))
print(f"Dataset de treino reduzido para {len(small_train_dataset)} exemplos.")

formatted_train_dataset = small_train_dataset.map(format_spider_for_training)

# run_training(training_args_1, lora_config_1, base_model, formatted_train_dataset, tokenizer)
run_training(training_args_2, lora_config_2, base_model, formatted_train_dataset, tokenizer)

print("Funções de treinamento com lógica de checkpoint definidas.")

Unused kwargs: ['load_in_8bit_fp32_cpu_offload']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.17s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Dataset de treino reduzido para 90 exemplos.


Map: 100%|██████████| 90/90 [00:00<00:00, 1857.22 examples/s]
/home/alexandre/nlp_metric/.venv/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/alexandre/nlp_metric/.venv/lib/python3.12/site-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/home/alexandre/nlp_metric/.venv/lib/python3.12/site-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/home/alexandre/nlp_metric/.venv/lib/python3.12/site-

Iniciando/Continuando treino para: ./results/config_2


Step,Training Loss
20,1.640900


Treino concluído. Modelo final salvo em ./results/config_2


/home/alexandre/nlp_metric/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Funções de treinamento com lógica de checkpoint definidas.


In [ ]:
def format_spider_for_training(example):
    return {
        "text": f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nConverta a seguinte pergunta para SQL: {example['question']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{example['query']}<|eot_id|>"
    }
model_name = "microsoft/phi-2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    llm_int8_threshold=6.0,
    load_in_8bit_fp32_cpu_offload=True  
)
base_model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
formatted_train_dataset = spider_dataset['train'].map(format_spider_for_training)

# -- CONFIGURAÇÃO 1 --
lora_config_1 = LoraConfig(r=16, lora_alpha=32, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")
training_args_1 = TrainingArguments(
    output_dir=OUTPUT_DIR_CONFIG_1, num_train_epochs=1, per_device_train_batch_size=4, 
    gradient_accumulation_steps=2, optim="paged_adamw_32bit", learning_rate=2e-4, 
    fp16=True, logging_steps=20, save_strategy="epoch"
)

# -- CONFIGURAÇÃO 2 (Hiperparâmetro alterado: num_train_epochs) 
lora_config_2 = LoraConfig(r=16, lora_alpha=32, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")
training_args_2 = TrainingArguments(
    output_dir=OUTPUT_DIR_CONFIG_2, num_train_epochs=2, per_device_train_batch_size=4, 
    gradient_accumulation_steps=2, optim="paged_adamw_32bit", learning_rate=2e-4, 
    fp16=True, logging_steps=20, save_strategy="epoch"
)

def run_training(training_args, lora_config):
    model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
    
    trainer = SFTTrainer(
        model=model, train_dataset=formatted_train_dataset, peft_config=lora_config, 
        dataset_text_field="text", max_seq_length=512, tokenizer=tokenizer, args=training_args
    )
    print(f"Iniciando treino para: {training_args.output_dir}")
    trainer.train()
    print(f"Treino concluído. Modelo salvo em {training_args.output_dir}")
    trainer.save_model(training_args.output_dir)

run_training(training_args_1, lora_config_1)
# run_training(training_args_2, lora_config_2)
print("Funções de treinamento definidas.")

Unused kwargs: ['load_in_8bit_fp32_cpu_offload']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.03s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.79s/it]
/home/alexandre/nlp_metric/.venv/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/alexandre/nlp_metric/.venv/lib/python3.12/site-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transf

Iniciando treino para: ./results_config1


Step,Training Loss


KeyboardInterrupt: 

In [4]:
import bitsandbytes as bnb
print(bnb.__version__)  # should be ≥0.39.0


0.45.2


## Fase 3: Avaliação de Desempenho na Tarefa-Alvo com Métrica Customizada

### 3.1 [IMPLEMENTADO] Métrica Customizada `ExecutionAccuracy`
A métrica customizada para DeepEval, que deve ser salva em um arquivo separado como `custom_metrics/execution_accuracy.py`.

In [ ]:
from deepeval.metrics import BaseMetric
from deepeval.test_case import LLMTestCase
import pytest

class ExecutionAccuracyMetric(BaseMetric):
    # A nota mínima para aprovação no DeepEval. Como nossa métrica é 0 ou 1.8, definimos como 1.8.
    threshold: float = 1.8

    def __init__(self, db_path_template: str):
        self.db_path_template = db_path_template

    def measure(self, test_case: LLMTestCase) -> float:
        # test_case.input contém a questão e test_case.context contém o db_id
        db_id = test_case.context[0]
        db_path = self.db_path_template.format(db_id=db_id)
        if not os.path.exists(db_path):
            self.success = False
            self.reason = f"Banco de dados não encontrado: {db_path}"
            return 0.0
        
        conn = sqlite3.connect(db_path) 
        cursor = conn.cursor()
        
        try:
            # Executa a consulta SQL gerada (actual_output)
            cursor.execute(test_case.actual_output)
            predicted_result = cursor.fetchall()
            
            # Executa a consulta ground truth (expected_output) 
            cursor.execute(test_case.expected_output)
            expected_result = cursor.fetchall()
            
            # Compara os conjuntos de resultados (insensível à ordem)
            if set(predicted_result) == set(expected_result):
                self.success = True
                return 1.8 
            else:
                self.success = False
                self.reason = f"Resultados divergentes. Esperado: {expected_result}, Obtido: {predicted_result}"
                return 0.0 # 
        except Exception as e:
            self.success = False
            self.reason = f"Erro de execução SQL: {e}"
            return 0.0
        finally:
            conn.close()

    def is_successful(self) -> bool:
        return self.success

    @property
    def __name__(self):
        return "Execution Accuracy"

print("Métrica 'ExecutionAccuracyMetric' definida.")

Métrica 'ExecutionAccuracyMetric' definida.


/home/alexandre/nlp_metric/.venv/lib/python3.12/site-packages/deepeval/__init__.py:41: UserWarning: You are using deepeval version 0.21.11, however version 3.1.8 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


### 3.2 [IMPLEMENTADO] Avaliação Automatizada
Lógica para carregar um modelo fine-tuned, gerar as predições e criar os Test Cases para o DeepEval.

In [5]:
def get_finetuned_model(adapter_path):
    """Carrega o modelo base e aplica o adaptador LoRA treinado."""
    model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
    model = PeftModel.from_pretrained(model, adapter_path)
    model = model.merge_and_unload() 
    return model

In [6]:
finetuned_model_1 = get_finetuned_model(OUTPUT_DIR_CONFIG_1)

Loading checkpoint shards: 100%|██████████| 2/2 [00:15<00:00,  7.96s/it]
/home/alexandre/nlp_metric/.venv/lib/python3.12/site-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.75s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:


def generate_predictions_for_deepeval(model, tokenizer, dataset):
    """Gera predições e formata como LLMTestCases."""
    test_cases = []
    for item in tqdm(dataset, desc="Gerando predições para DeepEval"):
        prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nConverta a seguinte pergunta para SQL: {item['question']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=128, pad_token_id=tokenizer.eos_token_id)
        
        full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
        assistant_response = full_output.split("assistant\n\n")[-1]
        
        test_case = LLMTestCase(
            input=item['question'],
            actual_output=assistant_response,
            expected_output=item['query'],
            context=[item['db_id']] 
        )
        test_cases.append(test_case)
    return test_cases
model_name = "microsoft/phi-2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    llm_int8_threshold=6.0,
    load_in_8bit_fp32_cpu_offload=True 
)
spider_dev_split = spider_dataset['validation']


# Gere os test cases (use uma amostra para agilizar)
test_cases_config_1 = generate_predictions_for_deepeval(finetuned_model_1, tokenizer, spider_dev_split.select(range(50)))


print("Funções para avaliação com DeepEval definidas. A execução é via CLI.")

Unused kwargs: ['load_in_8bit_fp32_cpu_offload']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Gerando predições para DeepEval: 100%|██████████| 50/50 [09:19<00:00, 11.18s/it]

Funções para avaliação com DeepEval definidas. A execução é via CLI.


In [10]:
import pickle


print(f"{len(test_cases_config_1)} casos de teste gerados.")

output_file = "test_cases_config_1.pkl"
with open(output_file, "wb") as f:
    pickle.dump(test_cases_config_1, f)

print(f"\nCasos de teste salvos em '{output_file}'.")
print("Para executar a avaliação, crie um script de teste e rode 'pytest' no terminal.")

50 casos de teste gerados.

Casos de teste salvos em 'test_cases_config_1.pkl'.
Para executar a avaliação, crie um script de teste e rode 'pytest' no terminal.


## Fase 4: Análise Quantitativa de Regressão de Capacidade

### 4.1 e 4.2 [IMPLEMENTADO] Metodologia de Avaliação MMLU e Cálculo de Acurácia
Implementação da avaliação 4-shot para o MMLU.

In [3]:
# Carregar MMLU
mmlu_dataset = load_dataset("cais/mmlu", 'all')


subjects = {
    'STEM': 'high_school_computer_science', 
    'Humanidades': 'formal_logic',  
    'Ciências Sociais': 'high_school_economics'
}

SEED = 42 

mmlu_suite = {}
for category, subject in subjects.items():
    filtered_dataset = mmlu_dataset['test'].filter(lambda x: x['subject'] == subject)
    
    if len(filtered_dataset) < 50:
        print(f"Aviso: Encontradas apenas {len(filtered_dataset)} questões para o assunto '{subject}'. Usando todas as disponíveis.")
        mmlu_suite[category] = filtered_dataset.shuffle(seed=SEED)
    else:
        mmlu_suite[category] = filtered_dataset.shuffle(seed=SEED).select(range(50))

print(f"Suíte MMLU criada com as categorias e número de questões:")
for category, data in mmlu_suite.items():
    print(f"- {category}: {len(data)} questões")
def format_mmlu_prompt(question_data, few_shot_examples):
    """Cria um prompt 4-shot para uma questão do MMLU."""
    prompt = "Responda a seguinte questão de múltipla escolha.\n\n"
    for ex in few_shot_examples:
        prompt += f"Questão: {ex['question']}\n"
        prompt += f"Opções: A) {ex['choices'][0]} B) {ex['choices'][1]} C) {ex['choices'][2]} D) {ex['choices'][3]}\n"
        prompt += f"Resposta: {['A', 'B', 'C', 'D'][ex['answer']]}\n\n"
    
    prompt += f"Questão: {question_data['question']}\n"
    prompt += f"Opções: A) {question_data['choices'][0]} B) {question_data['choices'][1]} C) {question_data['choices'][2]} D) {question_data['choices'][3]}\n"
    prompt += f"Resposta:"
    return prompt

def evaluate_mmlu(model, tokenizer, suite, dev_split):
    """Avalia um modelo na suíte MMLU usando 4-shot prompting."""
    results = {}
    few_shot_examples = dev_split.select(range(4))
    
    for category, dataset in suite.items():
    
        if len(dataset) == 0:
            print(f"Aviso: Nenhuma questão encontrada para a categoria '{category}'. Pulando a avaliação.")
            results[category] = 0.0 
            continue
        
        correct_predictions = 0
        for item in tqdm(dataset, desc=f"Avaliando {category}"):
            prompt = format_mmlu_prompt(item, few_shot_examples)
            inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=5, pad_token_id=tokenizer.eos_token_id)
            
            prediction_text = tokenizer.decode(outputs[0][-5:], skip_special_tokens=True).strip()
            predicted_char = prediction_text[0] if prediction_text else ''
            
            ground_truth_char = ['A', 'B', 'C', 'D'][item['answer']]
            
            if predicted_char.upper() == ground_truth_char:
                correct_predictions += 1
        
        accuracy = (correct_predictions / len(dataset)) * 100
        results[category] = accuracy
        
    # Filtra os valores None ou N/A antes de calcular a média
    valid_results = [res for res in results.values() if isinstance(res, (int, float))]
    if valid_results:
        results['Agregada'] = np.mean(valid_results)
    else:
        results['Agregada'] = 0.0

    return results

# Avaliar modelo base
baseline_mmlu_results = evaluate_mmlu(base_model, tokenizer, mmlu_suite, mmlu_dataset['dev'])

# Avaliar modelo fine-tuned
finetuned_model_1 = get_finetuned_model(OUTPUT_DIR_CONFIG_1)
# finetuned_mmlu_results_1 = evaluate_mmlu(finetuned_model_1, tokenizer, mmlu_suite, mmlu_dataset['dev'])
print("Função de avaliação MMLU definida.")

Aviso: Encontradas apenas 0 questões para o assunto 'high_school_economics'. Usando todas as disponíveis.
Suíte MMLU criada com as categorias e número de questões:
- STEM: 50 questões
- Humanidades: 50 questões
- Ciências Sociais: 0 questões


NameError: name 'base_model' is not defined

### 4.3 [IMPLEMENTADO] Análise de Regressão
Função para calcular e exibir a variação percentual de acurácia.

In [15]:
def report_regression_analysis(baseline_results: dict, finetuned_results: dict):
    """Calcula e imprime a análise de regressão de capacidade."""
    print("\n--- Análise de Regressão de Capacidade (MMLU) ---")
    
    # Análise agregada [cite: 43]
    agg_baseline = baseline_results['Agregada']
    agg_finetuned = finetuned_results['Agregada']
    agg_change = ((agg_finetuned - agg_baseline) / agg_baseline) * 100 if agg_baseline > 0 else 0
    print(f"Acurácia Agregada (Base): {agg_baseline:.2f}%")
    print(f"Acurácia Agregada (Fine-tuned): {agg_finetuned:.2f}%")
    print(f"Variação Percentual Agregada: {agg_change:.2f}%\n")

    # Análise por categoria [cite: 43]
    for category in subjects.keys():
        base = baseline_results[category]
        ft = finetuned_results[category]
        change = ((ft - base) / base) * 100 if base > 0 else 0
        print(f"Categoria: {category}")
        print(f"  Acurácia Base: {base:.2f}% | Acurácia Fine-tuned: {ft:.2f}% | Variação: {change:.2f}%\n")

# -- Exemplo com dados fictícios. Substitua com seus resultados reais --
baseline_results_example = {'STEM': 75.0, 'Humanidades': 80.0, 'Ciências Sociais': 78.0, 'Agregada': 77.67}
finetuned_results_example = {'STEM': 70.0, 'Humanidades': 72.0, 'Ciências Sociais': 71.0, 'Agregada': 71.0}

report_regression_analysis(baseline_results_example, finetuned_results_example)


--- Análise de Regressão de Capacidade (MMLU) ---
Acurácia Agregada (Base): 77.67%
Acurácia Agregada (Fine-tuned): 71.00%
Variação Percentual Agregada: -8.59%

Categoria: STEM
  Acurácia Base: 75.00% | Acurácia Fine-tuned: 70.00% | Variação: -6.67%

Categoria: Humanidades
  Acurácia Base: 80.00% | Acurácia Fine-tuned: 72.00% | Variação: -10.00%

Categoria: Ciências Sociais
  Acurácia Base: 78.00% | Acurácia Fine-tuned: 71.00% | Variação: -8.97%

